In [7]:
# runs on docker2:v1 w/ the speed and volume *.xlsx files

import numpy as np
import os
import pandas as pd
import re

In [8]:
path = os.getcwd()
print(path)
#/opt/dssg-hot/data/speed_volume/pythonscripts

/opt/dssg-hot/data/speed_volume/pythonscripts


In [9]:
fpathbase = '/opt/dssg-hot/data/shirleydata/speed_volume_extra/xlsxfiles'
fpaths = sorted([f for f in os.listdir(fpathbase)])
print(fpaths)

['405 N Speed GP 13.6-28.98 M-F 2018 _1m', '405 N Speed HOV 13.6-28.98 M-F 2018 _1m', '405 N Volume per Hour GP 13.6-28.98 M-F 2018 _1m', '405 N Volume per Hour HOV 13.6-28.98 M-F 2018 _1m', '405 S Speed GP 13.92-28.98 M-F 2018 _1m', '405 S Speed HOV 13.92-28.98 M-F 2018 _1m', '405 S Volume per Hour GP 13.92-28.98 M-F 2018 _1m', '405 S Volume per Hour HOV 13.92-28.98 M-F 2018 _1m']


In [10]:
# get the entry plaza numbers from loop mileposts closest to the 0.5 mi intervals we downloaded data for
nb_entry_plu = pd.read_csv('/opt/dssg-hot/data/milepost_lookups/nb_mileposts_entries_plaza_lookup.csv')
sb_entry_plu = pd.read_csv('/opt/dssg-hot/data/milepost_lookups/sb_mileposts_entries_plaza_lookup.csv')

In [11]:
nb_entry_plu['Loop mile post now'] = nb_entry_plu['Mile post (SRweb)']
nb_entry_plu['Loop mile post now'].fillna(nb_entry_plu['Loop mile post to take'], inplace=True)
sb_entry_plu['Loop mile post now'] = sb_entry_plu['Mile post (SRweb)']
sb_entry_plu['Loop mile post now'].fillna(sb_entry_plu['Loop mile post to take'], inplace=True)

In [12]:
nb_entry_plu.head()

,Entry plaza name,Entry plaza number,Actual entry name,Mile post (SRweb),Loop mile post before entry,Loop mile post after entry,Loop mile post to take,"Diff btwn loop mile and actual mile post (pos = take before, neg = take after)",Closest 0.5 mi mp to SRweb mp (or loop mp to take when SRweb mp is missing),Notes,Loop mile post now
0,NB01,3,NB Entry 1,13.74,13.60,13.92,13.60,0.14,13.5,NaN,13.74
1,NB02,4,NB Entry 2 (end of on ramp),13.93,13.92,14.27,13.92,0.01,14.0,NaN,13.93
2,NB03,5,NB Entry 3/Exit 1 (midpt),15.76,15.63,16.12,15.63,0.13,16.0,NaN,15.76
3,NB04,6,NB Entry 4/Exit 2 (midpt),19.12,18.70,19.21,19.21,-0.09,19.0,NaN,19.12
4,NB05,7,NaN,NaN,NaN,NaN,20.22,NaN,20.0,Shouldn't exist as entry --> arbitrarily chose...,20.22


In [13]:
sb_entry_plu.head(6)

,Entry plaza name,Entry plaza number,Actual entry,Mile post (SRweb),Loop mile post before entry,Loop mile post after entry,Loop mile post to take,"Diff btwn loop mile and actual mile post (pos = take before, neg = take after)",Closest 0.5 mi mp to SRweb mp (or loop mp to take when SRweb mp is missing),Notes,Loop mile post now
0,SB01,13,SB Entry 1,28.89,28.98,28.53,28.98,0.09,29.0,NaN,28.89
1,SB02,14,SB Entry 2/Exit 1 (midpt),27.61,27.96,27.44,27.44,-0.16,27.5,NaN,27.61
2,SB03,15,SB Entry 3/Exit 2 (midpt),25.95,26.16,25.68,26.16,0.21,26.0,NaN,25.95
3,SB04,16,SB Entry 4 (midpt),22.50,22.73,22.31,22.31,-0.19,22.5,NaN,22.50
4,SB05,17,SB Entry 5,21.59,21.83,21.36,21.36,-0.23,21.5,NaN,21.59
5,SB06,18,NaN,NaN,NaN,NaN,20.93,NaN,21.0,Shouldn't exist as entry --> arbitrarily chose...,20.93


In [8]:
merged_speed = pd.DataFrame()
merged_volume = pd.DataFrame()

# all following files are already pruned to only include those w/ 0.5 mile posts that
# are closest to our actual entries (this is a weird way to do it, but ok - would've
# been better to just keep all the xlsx files at each 0.5 mi interval and then find
# the closest 0.5 mi interval to each actual entry w/ that entry's assoc plaza)
for fpath in fpaths:

    files = sorted([f for f in os.listdir(fpathbase + '/' + fpath) if f.endswith('.xlsx')])
    print('folder: ')
    print(fpathbase + '/' + fpath)
    print('files in folder: ')
    print(files)
    
    for file in files:
        
        # get whether it's GP or HOV; lt = lane type
        lt_str = 'GP' if 'GP' in file else 'HOV'

        # get the file's milepost
        mp_start_stridx = file.find('.xlsx')-4
        mp_end_stridx = file.find('.xlsx')
        mp_str = file[mp_start_stridx:mp_end_stridx]

        # get whether it's NB or SB
        dir_str = 'NB' if '405 N' in file else 'SB'

        # get corresponding closest entry plaza --> column name
        if dir_str=='NB':
            entry_plaza_str = \
                str(nb_entry_plu['Entry plaza number']
                .iloc[abs(nb_entry_plu['Loop mile post now']-float(mp_str)).idxmin()])\
                .zfill(2)
        elif dir_str=='SB':
            entry_plaza_str = \
                str(sb_entry_plu['Entry plaza number']\
                .iloc[abs(sb_entry_plu['Loop mile post now']-float(mp_str)).idxmin()])\
                .zfill(2)
        colname0 = lt_str + '_' + dir_str + '_' + mp_str
        colname = lt_str + '_' + entry_plaza_str

        # make sure everything parsed + looked up correctly
        print('colname: ')
        print(colname0 + ' --> ' + colname)

        # get speeds/volumes from current file --> put them into one column
        data = pd.read_excel(fpathbase + '/' + fpath + '/' + file, 'Fixed interval data')
        data.rename(columns={'Unnamed: 0':'Time of day'}, inplace=True)
        melt = pd.melt(data, id_vars=['Time of day'])
        melt['Time']=melt.apply(lambda r : 
               pd.datetime.combine(r['variable'],r['Time of day']),1)
        melt.drop(columns=['Time of day','variable'], inplace=True)
        melt.rename(columns={'value':colname},inplace=True)
        melt.set_index('Time',inplace=True)

        # add latest file's column to final df called merged + check w/ printing
        if 'Speed' in file:
            if merged_speed.empty:
                merged_speed = melt
            else:
                merged_speed = pd.merge(merged_speed, melt, how='inner', left_index=True, right_index=True)
            
            if file==files[-1]:
                print('merged_speed df check: ')
                print(merged_speed.head())
                
        elif 'Volume' in file:
            if merged_volume.empty:
                merged_volume = melt
            else:
                merged_volume = pd.merge(merged_volume, melt, how='inner', left_index=True, right_index=True) 
            
            if file==files[-1]:
                print('merged_volume df check: ')
                print(merged_volume.head())

folder: 
../xlsxfiles/405 N Speed GP 13.6-28.98 M-F 2018 _1m
files in folder: 
['405 N Speed GP 13.6-28.98 M-F 2018 _1m 13.5.xlsx', '405 N Speed GP 13.6-28.98 M-F 2018 _1m 14.0.xlsx', '405 N Speed GP 13.6-28.98 M-F 2018 _1m 16.0.xlsx', '405 N Speed GP 13.6-28.98 M-F 2018 _1m 19.0.xlsx', '405 N Speed GP 13.6-28.98 M-F 2018 _1m 20.0.xlsx', '405 N Speed GP 13.6-28.98 M-F 2018 _1m 21.0.xlsx', '405 N Speed GP 13.6-28.98 M-F 2018 _1m 22.0.xlsx', '405 N Speed GP 13.6-28.98 M-F 2018 _1m 25.5.xlsx', '405 N Speed GP 13.6-28.98 M-F 2018 _1m 28.0.xlsx', '405 N Speed GP 13.6-28.98 M-F 2018 _1m 28.5.xlsx']
colname: 
GP_NB_13.5 --> GP_03
colname: 
GP_NB_14.0 --> GP_04
colname: 
GP_NB_16.0 --> GP_05
colname: 
GP_NB_19.0 --> GP_06
colname: 
GP_NB_20.0 --> GP_07
colname: 
GP_NB_21.0 --> GP_08
colname: 
GP_NB_22.0 --> GP_09
colname: 
GP_NB_25.5 --> GP_10
colname: 
GP_NB_28.0 --> GP_11
colname: 
GP_NB_28.5 --> GP_12
merged_speed df check: 
                     GP_03  GP_04  GP_05  GP_06  GP_07  GP_08  GP_

In [9]:
merged_speed.to_csv(r'/opt/dssg-hot/data/shirleydata/speed_volume_extra/concatenated_405_speeds_all_times_1min_no_duplicates.csv', index = True, header = True)
merged_volume.to_csv(r'/opt/dssg-hot/data/shirleydata/speed_volume_extra/concatenated_405_volumes_all_times_1min_no_duplicates.csv', index = True, header = True)

In [16]:
# SB10 and SB11 exits are the same loop --> create duplicate columns as follows:
# GP_22 = GP_23, HOV_22 = HOV_23
def duplicate22(merged):
    filter_cols = [col for col in merged if col.endswith('22')]
    print(filter_cols)
    new_cols = [colname.replace('22', '23') for colname in filter_cols]
    print(new_cols)
    merged[new_cols] = merged[filter_cols]
    print(merged.head())
    return merged

In [ ]:
merged_speed = duplicate22(merged_speed)
merged_volume = duplicate22(merged_volume)

In [ ]:
def take_5am_to_8pm_only(merged):
    merged['Time'] = merged.index
    merged['hour'] = merged['Time'].apply(lambda t: t.hour)
    merged = merged[merged['hour'].between(5, 19, inclusive=True)]
    merged.drop('hour', axis=1, inplace=True)
    merged.drop('Time', axis=1, inplace=True)
    print(merged.head())
    return merged

merged_speed = take_5am_to_8pm_only(merged_speed)
merged_volume = take_5am_to_8pm_only(merged_volume)

In [18]:
merged_speed.to_csv(r'/opt/dssg-hot/data/shirleydata/speed_volume_extra/concatenated_405_speeds_5am_to_8pm_1min.csv', index = True, header = True)
merged_volume.to_csv(r'/opt/dssg-hot/data/shirleydata/speed_volume_extra/concatenated_405_volumes_5am_to_8pm_1min.csv', index = True, header = True)

In [ ]:
# SAMPLE CODE TO REORGANIZE COLS TO BE GP THEN HOV, GP THEN HOV, ETC.:

# a = list(range(0,13))
# b = list(range(13,26))
# c = a + b
# c[::2] = a
# c[1::2] = b
# c
# new_order = [3,2,1,4,5,0]
# print(df[df.columns[new_order]]) 